
# Module 3: Trident Configuration

## Exercise 1: Working with Trident

**Objectives**

This exercise focuses on enabling you to do the following:
  - Create a NAS back end
  - Create a storage class for a NAS back end
  - Provision a persistent volume claim with a NAS back end
  - Mount the volumes in a pod
  - Perform back-end management by using the tridentctl tool
  - Perform back-end management by using the kubectl tool
  - Configure customized naming conventions
  - Create a NAS economy back end
  - Provision NVMe namespaces using NetApp Trident

**Exercise Equipment**

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |




**Prerequisites**

Before starting this exercise, you should take the following actions:

  - Set up your Integrated Development Environment (IDE)
  - Download the courseware GIT repository
  - Configure your IDE to have access to your Kubernetes clusters
  - Create svm0
  - Configure svm0 to use the NFS v3 protocol
  - Install Trident in your source Kubernetes cluster
  - Ensure iSCSI, NVMe and NFS are properly configured on your worker nodes in the source Kubernetes cluster


---
---

#### Task 1: Create a NAS back end

In this task, you create a back end that uses ONTAP NAS storage driver. 

You can create many other back ends. 

For more information, see https://docs.netapp.com/us-en/trident/trident-use/backends.html.

**NOTE**: 

When you unzip the trident tar file, the path trident-installer/sample-inputs provides many examples of configuration files.


---

To complete this task successfully, you must first complete Task 4 in the Module 1
exercise. 

In that task, you configure a storage VM (storage virtual machine, also known as SVM) for the NFS protocol.

---

If you are using terminal to execute kubectl commands, please change directories in your terminal to Exercise 3 folder.


In [ ]:
pwd

---

Modify the [exercise3Task1.json](./exercise3Task1.json) file to add the appropriate settings;

Save the file as `exercise3Task1mod.json`:

  - Version: 1
  - Storage driver name: ontap-nas
  - Back-end name: c1-svm0-nfs-tbe
  - Management LIF: 192.168.0.30
  - Data LIF: 192.168.0.31
  - SVM: svm0
  - Username: vsadmin
  - Password: Netapp1!


<details> <summary>Solution  </summary>

[exercise3Task1.json](./Solutions/exercise3Task1.json)

```json
{
    "version": 1,
    "storageDriverName": "ontap-nas",
    "backendName": "c1-svm0-nfs-tbe",
    "managementLIF": "192.168.0.30",
    "dataLIF": "192.168.0.31",
    "svm": "svm0",
    "username": "vsadmin",
    "password": "Netapp1!"
}

---

The back-end definition is the only place that stores the credentials in plain text.

After you create the back end, usernames and passwords are encoded with Base64 and stored as Kubernetes secrets. 

Creating and updating a back end are the only operations that require knowledge of the credentials. 

These operations should be admin-only.



---

Verify that you are in the working directory where the tridentctl tool and the back-end JSON are present.


---


Create the back end by using the tridentctl tool:

`tridentctl -n trident create backend -f exercise3Task1mod.json`

Sample output:

```terminal
+---------------------+----------------+--------------------------------------+--------+------------+--------+
| NAME | STORAGE DRIVER | UUID | STATE | USER-STATE | VOLUMES|
+---------------------+----------------+--------------------------------------+--------+------------+--------+
| c1-svm0-nfs-tbe | ontap-nas | 804a70a5-5959-435d-a9c7-6357230f2e13 | online | normal | 0 |
+---------------------+----------------+--------------------------------------+--------+------------+--------+
```


---

In [ ]:
tridentctl -n trident create backend -f exercise3Task1mod.json


---

Review the tridentctl logs:


In [ ]:
tridentctl -n trident logs | tail -n 10


---
---

#### Task 2: Create a Storage Class for a NAS back end

In this task, you create a storage class that uses the NAS back end that you created in Task 1.


---

In your integrated development environment (IDE), open the [exercise3Task2.yaml](./exercise3Task2.yaml) file.


---

Add the correct `backendType` to the parameters.

This value is the `storageDriverName` from the back-end JSON.

<details> <summary> Solution </summary>
  
[exercise3Task2.yaml](./Solutions/exercise3Task2.yaml)

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: c1-svm0-nfs-sc
provisioner: csi.trident.netapp.io
parameters:
  backendType: ontap-nas
  storagePools: "c1-svm0-nfs-tbe:.*"

---

Add the correct storagePools:

` "c1-svm0-nfs-tbe.*"` 

and save the file `./exercise3Task2mod.yaml`.


---

Create the storage class:


In [ ]:
kubectl create -f exercise3Task2mod.yaml


---

Verify that the storage class is created:


In [ ]:
kubectl get sc c1-svm0-nfs-sc


---

Review the storage class by using the tridentctl tool:


In [ ]:
tridentctl -n trident get storageclass c1-svm0-nfs-sc -o json


---
---

#### Task 3: Provision a Persistent Volume Claim with a NAS back end

In this task, you create a persistent volume claim (PVC) for a volume that uses the storage class that you created.


---

In your IDE, open the [exercise3Task3.yaml](./exercise3Task3.yaml) file.


---

Update the `storageClassName` with the name of the storage class that you created in the previous task,

and then save the file as ./exercise3Task3mod.yaml.

<details> <summary>Solution  </summary>  

[exercise3Task3.yaml](./Solutions/exercise3Task3.yaml)

```yaml
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: c1-svm0-nfs-pvc-1
  namespace: default
  annotations:
    trident.netapp.io/snapshotDirectory: "true"
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: c1-svm0-nfs-sc


---

Create the PVC for a pod to use later:


In [ ]:
kubectl create -f exercise3Task3mod.yaml


---

After a moment, verify that you created the PVC:

Sample output:

```terminal
NAME STATUS VOLUME CAPACITY ACCESS MODES STORAGECLASS …
c1-svm0-nfs-pvc-1 Bound pvc-bf08cf3c-1a31… 1Gi RWO c1-svm0-nfs-sc …



In [ ]:
kubectl -n default get pvc c1-svm0-nfs-pvc-1


---

Navigate to ONTAP System Manager, and see the new volume that Trident created:

https://192.168.0.101/sysmgr/v4/storage/volumes.



---
---

#### Task 4: Mount the volumes in a pod

In this task, you create a NGINX pod that uses the persistent volume (PV) and creates a default webpage in the PV. 

This task includes a challenge step that asks you to expose the pod by using a NodePort service 
and then view your custom webpage.


In your IDE, review the [exercise3Task4-1.yaml](./exercise3Task4-1.yaml) file.



---

Set the claimName definition to the name of the PVC that you created in the previous task.

<details> <summary>  Solution</summary>  

[exercise3Task4-1.yaml](./Solutions/exercise3Task4-1.yaml)

```yaml
kind: Pod
apiVersion: v1
metadata:
  name: nfs-pod
  namespace: default
  labels:
    app: nfs-web
spec:
  volumes:
    - name: nfs-storage
      persistentVolumeClaim:
       claimName: c1-svm0-nfs-pvc-1
  containers:
    - name: nfs-container
      image: nginx:1.25-alpine-slim
      imagePullPolicy: IfNotPresent
      resources:
        requests:
          memory: "64Mi"
          cpu: "250m"
        limits:
          memory: "128Mi"
          cpu: "500m"
      ports:
        - containerPort: 80
          name: "http-server"
      volumeMounts:
        - mountPath: "/usr/share/nginx/html"
          name: nfs-storage

---

Save the file as exercise3Task4-1mod.yaml



---

Create the pod to use the Trident volume:


In [ ]:
kubectl create -f exercise3Task4-1mod.yaml


---

Verify that you created the pod:


In [ ]:
kubectl -n default get pod nfs-pod


Connect to the pod:

View how the PVC is mounted in the container:

`# df -h`

In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "df -h"


In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "df -h" |grep -iA1 trident


---

Change the directory to the Trident persistent volume:

`# cd /usr/share/nginx/html`

Create an HTML file in the current directory:
`# echo '<html><body>Hello [your name] using NFS</body></html>' > index.html`



---

In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "echo '<html><body>Hello nfs-pod using NFS</body></html>' >  /usr/share/nginx/html/index.html;"


In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "cat  /usr/share/nginx/html/index.html"



---

Create a NodePort service and view the webpage contains your custom message:

[exercise3Task4-2.yaml](./exercise3Task4-2.yaml)


In [ ]:

kubectl create -f exercise3Task4-2.yaml

---

View the services:

`kubectl -n default get services`


Sample output:
```terminal
NAME TYPE CLUSTER-IP EXTERNAL-IP PORT(S) AGE
kubernetes ClusterIP 10.96.0.1 <none> 443/TCP 2d
nfs-web NodePort 10.106.85.27 <none> 80:31319/TCP 4m39s
```

In [ ]:
kubectl -n default get services

Open a browser to one of the Kubernetes cluster node IP addresses and the NodePort referenced in the previous step. 

For example: http://192.168.0.62:31319. 

You should see your index.html page displayed in the browser. NOTE: Use HTTP.


In [ ]:
nodeport=$(kubectl describe service nfs-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

In [ ]:
kubectl describe service nfs-web

---
---

#### Task 5: Perform Back-End management by using the tridentctl tool

In this task, you investigate the tridentctl commands and delete the pod that hosted your custom webpage. 

You then re-create the pod and notice that the persistent volume, which reattached to the new pod and your custom webpage, has persisted.


---

jQuery is a json query tool which is installed on your jumphost:

it can be installed using

`sudo apt install -y jq`


---

Use the tridentctl tool to identify the storage class that is mapped to the correct backend 

(for your convenience, you can copy this command from the [exercise3Task5.txt](./exercise3Task5.txt) file):

tridentctl get backend -n trident -o json | jq '[.items[] | {backend: .name, storageClasses: [.storage[].storageClasses]|unique}]'


Sample output:
```json
[
  { "backend": "c1-svm0-nfs-tbe",
    "storageClasses": [
     [
       "c1-svm0-nfs-sc"
     ]
    ]   
  }
]


In [ ]:
tridentctl get backend -n trident -o json | jq '[.items[] | {backend: .name, storageClasses: [.storage[].storageClasses]|unique}]'

---

You can delete and update the back end by using the tridentctl tool. 

For more information, 

see https://docs.netapp.com/us-en/trident/trident-use/backend_ops_tridentctl.html#create-a-backend.


---

Delete the NFS-supported pod:


In [ ]:
kubectl -n default delete pod nfs-pod


---

Investigate the logs and see if the volume was deleted when the pod was deleted:


In [ ]:
tridentctl logs -n trident | tail -n 20


---

Notice that the volume was just “unpublished” and answer the following questions: 

How would you delete the volume automatically when you delete the pod? 

Does the PVC still exist? 

Also  notice the finalizer that is associated with the PVC.


---

Navigate to ONTAP System Manager and notice that the volume that Trident created for the
NFS-based PVC is still there: 

https://192.168.0.101/sysmgr/v4/storage/volumes.


In [ ]:
ssh admin@cluster1 volume show

---

Re-create the pod to use the Trident NFS-provided volume:



In [ ]:
kubectl create -f exercise3Task4-1mod.yaml


---

Display the webpage and verify the webpage contains your custom message.


In [ ]:
nodeport=$(kubectl describe service nfs-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

---

Do not destroy any objects. You use the objects in the next exercise.


---
---

#### Task 6: Perform Back-End management by using the kubectl tool

Previously, you created a back end by using the tridentctl tool. 

In this task, you create a back end by using the `TridentBackendConfig` custom resource (CR) with the credentials that are stored in a Kubernetes secret. 

For this task, you create a new SVM with the iSCSI protocol configured. 

NOTE:

SVMs allow multiple protocols. 

You could add the iSCSI configuration to svm0, but you create another SVM in this task to keep the two SVMs functionality separate. 

You also use the SVM administrator (vsadmin) and a separate management path for Trident to communicate with the SVMs.


---

Verify that you created the gateway-system namespace and that the operator pod is running in that namespace.


---

Review and execute the [exercise3Task6-1.yaml](./exercise3Task6-1.yaml) file to create an SVM called svm1 with the defined protocol in ONTAP Cluster 1. 

NOTE: 


Execute this file with a kubectl apply command, otherwise, an error will occur stating that the cluster1 admin’s secret is already created.


---

In [ ]:
kubectl apply -f exercise3Task6-1.yaml

CHALLENGE STEP: 

Review the logs of the manager container for the gateway-manager deployment’s pod to see the gateway operator in action:


`kubectl -n gateway-system logs gateway-operator-[unique id] -c manager`


In [ ]:
go=$(kubectl -n gateway-system get pods|grep gateway-operator| awk '{print $1}');echo $go



In [ ]:
kubectl -n gateway-system logs $go -c manager

---

Open a browser and go to https://192.168.0.101/

(which is your Cluster1 management LIF’s address).
 

---

For now, use the standard System Manager. 

Click the link: Not now. Sign in to System Manager. 

Skip this step if you already have selected System Manager instead of using NetApp BlueXP.


---

Authenticate with your ONTAP cluster by providing the following credentials:
 
 - Login as: admin
 
 - Password: Netapp1!


---


Click Sign In.


---

From the left pane, navigate to Storage > Storage VMs.


---

Review the settings of svm1 and verify that iSCSI and NVMe/TCP are configured.
Also verify that the Cluster1_01_FC_1 is an available local tier for this SVM.











In [ ]:
ssh admin@cluster1  "vserver context svm1; vserver show -fields aggr-list;nvme show -instance;iscsi show -instance"

---

Update the [exercise3Task6-2.yaml](./exercise3Task6-2.yaml) file with the details of the iSCSI SVM:
  
  - User name: vsadmin
  
  - Password: Netapp1!
  
  - Management LIF: 192.168.0.40
  
  - SVM: svm1

Save the file as `exercise3Task6-2mod.yaml`

NOTE: 

Generally, you should not specify a Data LIF for block protocols, otherwise, multipath 
would be disabled.

<details> <summary>  Solution</summary>  

[exercise3Task6-2.yaml](./Solutions/exercise3Task6-2.yaml)

---

Create the secret and the back end by using the kubectl tool:

In [ ]:
kubectl create -f exercise3Task6-2mod.yaml

---

In the Kubernetes IDE extension, ensure you are in the trident namespace. 

Navigate to **Clusters** > **source-admin@source** > **Custom Resources** >
**tridentbackendconfigs** > **c1-svm1-iscsi-tbc**.

This back end is the one that you created. 

The status should show the last operation status as success and the phase as bound.


---

Verify that you created the back-end configuration:


In [ ]:
kubectl -n trident get tbc -o wide


---

Get details on the back-end configuration that you created:


In [ ]:
kubectl -n trident describe tbc c1-svm1-iscsi-tbc


---

Review and update the name of the back end in the YAML, and then create the storage class
in the [exercise3Task6-3.yaml](./exercise3Task6-3.yaml) file:

Save the file as **exercise3Task6-3mod.yaml**

<details> <summary>  Solution</summary>

[exercise3Task6-3.yaml](./Solutions/exercise3Task6-3.yaml)

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: c1-svm1-iscsi-sc
provisioner: csi.trident.netapp.io
parameters:
  backendType: ontap-san
  storagePools: "c1-svm1-iscsi-tbe:.*"

In [ ]:
kubectl create -f exercise3Task6-3mod.yaml


---

Review and update the storage class name in the YAML, and then create the PVC in
[exercise3Task6-4.yaml](./exercise3Task6-4.yaml):


<details> <summary>  Solution</summary>

[exercise3Task6-4.yaml](./Solutions/exercise3Task6-4.yaml)  

```yaml
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: c1-svm1-iscsi-pvc-1
  namespace: default
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: c1-svm1-iscsi-sc

In [ ]:
kubectl create -f exercise3Task6-4mod.yaml


---

Navigate to ONTAP System Manager and see the new volume that Trident created:



https://192.168.0.101/sysmgr/v4/storage/volumes.


In [ ]:
ssh admin@cluster1 "volume show -vserver svm1"

---

Navigate to the LUNs in ONTAP System Manager and see the new LUN that Trident created:



https://192.168.0.101/sysmgr/v4/storage/luns.


In [ ]:
ssh admin@cluster1 "lun show -vserver svm1"

---

Review and update the claim name in the YAML, and then create the pod in the
[exercise3Task6-5.yaml](./exercise3Task6-5.yaml) file:

Save the file as **exercise3Task6-5mod.yaml**


<details> <summary>  Solutions</summary>

[exercise3Task6-5.yaml](./Solutions/exercise3Task6-5.yaml)  

```yaml
kind: Pod
apiVersion: v1
metadata:
  name: san-pod
  namespace: default
  labels:
    app: san-web
spec:
  volumes:
    - name: san-storage
      persistentVolumeClaim:
       claimName: c1-svm1-iscsi-pvc-1
  containers:
    - name: san-container
      image: nginx:1.25-alpine-slim
      resources:
        requests:
          memory: "64Mi"
          cpu: "250m"
        limits:
          memory: "128Mi"
          cpu: "500m"
      ports:
        - containerPort: 80
          name: "http-server"
      volumeMounts:
        - mountPath: "/usr/share/nginx/html"
          name: san-storage

In [ ]:
kubectl create -f exercise3Task6-5mod.yaml


---

Verify that you created the pod:


In [ ]:
kubectl -n default get pod san-pod


---

Connect to the pod:


`kubectl -n default exec -it san-pod -- /bin/sh`

---

View how the PVC is mounted in the container:

`# df -h`



---

Change the directory to the Trident persistent volume:

`# cd /usr/share/nginx/html`



---

Create a file in this location:

`# echo '<html><body>Hello [your name] using iSCSI</body></html>' > index.html`



---

Use Ctrl-D to exit the container’s shell.


In [ ]:
kubectl -n default exec -it san-pod -- /bin/sh -c "df -h"


In [ ]:
kubectl -n default exec -it san-pod -- /bin/sh -c "df -h" |grep -iA1 00a098


---

In [ ]:

kubectl -n default exec -it san-pod -- /bin/sh -c "echo '<html><body>Hello san-pod using iSCSI</body></html>' >  /usr/share/nginx/html/index.html;"


---

CHALLENGE STEP: 

Create a NodePort service and view the custom webpage. 

Note:

Check the default namespace you selected in IDE Kubernetes Extension

An example of this can be found in Solutions/exercise3Task6-6.yaml.

<details> <summary>Solution  </summary> 

[exercise3Task6-6.yaml](./Solutions/exercise3Task6-6.yaml) 

```yaml
apiVersion: v1
kind: Service
metadata:
  name: san-web
  namespace: default
spec:
  type: NodePort
  ports:
    - port: 80
      targetPort: 80
  selector:
    app: san-web

In [ ]:
kubectl apply -f exercise3Task6-6mod.yaml

In [ ]:
nodeport=$(kubectl -n default describe service san-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

---

CHALLENGE STEP: 

Delete the pod and verify that the LUN still exists.


In [ ]:
kubectl -n default delete pod san-pod

In [ ]:
ssh admin@cluster1 lun show -vserver svm1

---

CHALLENGE STEP: 

Re-create the pod and view the webpage that contains your custom
message.


In [ ]:
nodeport=$(kubectl -n default describe service san-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

In [ ]:
ssh admin@cluster1 lun show -vserver svm1

---

Do not destroy any objects. You use the objects in a later exercise.


---
---

#### Task 7: Configure customized naming conventions

In this task, you will work with a custom naming convention for a new `TridentBackendConfig` definition.

---

Within ONTAP System Manager, review the volumes names created by Trident by default.

Notice that default naming conventions for volumes looks something like this:

`trident_pvc_e018e7ab_a95b_4cb7_a366_85953d8fdec5`



---

Review the [exercise3Task7-1.yaml](./exercise3Task7-1.yaml) file.

Notice the following:

1. This single file creates a secret for cluster1’s admin credentials and then uses that secret in `TridentBackendConfig`.

2. In the `TridentBackendConfig` (tbc), the name of the tbc object is different than the backend. 
This is not necessary. We are just demonstrating that the tbc object and the backend can be different.

3. The storageclass’ storagepools is linked to the backend name designated in the tbc object.

---

Edit and save the [exercise3Task7-1.yaml](./exercise3Task7-1.yaml) file  as **exercise3Task7-1mod.yaml**  and replace the *change_me* in the following locations:

- nameTemplate: '{{ .labels.cluster }}_{{ .volume.Namespace }}_{{.volume.RequestName }}_{{ .config.BackendName }}'

- cluster: Cluster1

Note the following:

1. The labels.cluster in the name template will map to Cluster1 that we defined in the cluster label.

2. The volume namespace will be the persistent volume claim’s namespace while the volume RequestName will be name of the persistent volume claim.

3. Finally, the TridentBackendConfig’s BackendName will be appended to the end of the volume name.

<details> <summary>Solution  </summary>

[exercise3Task7-1.yaml](./Solutions/exercise3Task7-1.yaml)

```yaml

---

Create the secret, TridentBackendConfig and storageclass:


In [ ]:

kubectl create -f exercise3Task7-1mod.yaml


---


Next, create a pvc that uses the storageclass you created in this task by updating the *change_me* field in [exercise3Task7-2.yaml](./exercise3Task7-2.yaml). 

Save the files as **exercise3Task7-2mod.yaml** and then execute the file:


<details> <summary>Solution</summary>

[exercise3Task7-2.yaml](./Solutions/exercise3Task7-2.yaml)

```yaml

In [ ]:
kubectl create -f exercise3Task7-2mod.yaml


---

Review the PVC and PV created by exercise3Task7-2.yaml.


In [ ]:
kubectl -n default get pvc  

In [ ]:
kubectl -n default get pv

---

Within ONTAP System Manager, discover the name of the volume created by
exercise3Task7-2.yaml.

NOTE: 

Trident automatically adds a suffix corresponding to a slice of the volume’s UUID (a part of the volume.Name).


In [ ]:
ssh admin@cluster1 volume show -vserver svm0

---

CHALLENGE STEP: 

Experiment adding or replacing the name template in the TridentBackendConfig with other naming conventions such:

- {{ .config.StoragePrefix }}

- {{ slice .volume.Name }}

- Additional labels



---

CHALLENGE STEP: 

Discover the ONTAP volume name in the persistent volume’s internal
name attribute.


---

CHALLENGE STEP: 

Discover the default storagePrefix value if the TridentBackend (tbe) object
has {} as the value config.storage[0].ontap_config.storagePrefix.

Try:

`kubectl -n trident get tbe [some_tbe] -o jsonpath={".config.ontap_config.storage[0].defaults.nameTemplate"};echo`

In [ ]:
kubectl -n trident get tbe

In [ ]:
kubectl -n trident get tbe tbe-vrts8 -o jsonpath={".config.ontap_config.storage[0].defaults.nameTemplate"};echo

In [ ]:
kubectl -n default get pvc 

---
---

#### Task 8: Create a NAS economy back end

In this task, you will create a TridentBackendConfig to provisions qtrees instead of volumes, a storage class to use that backend configuration, and then a persistent volume claim to generate storage.



---

Review and update the *change_me* field in [exercise3Task8-1.yaml](./exercise3Task8-1.yaml).

Save the file as **exercise3Task8-1mod.yaml**


Create the secret, TridentBackendConfig and storageclass:

<details> <summary> Solution </summary> 

[exercise3Task8-1.yaml](./Solutions/exercise3Task8-1.yaml)
 
```yaml

In [ ]:
kubectl apply -f exercise3Task8-1mod.yaml


---


Next, create a pvc that uses the storageclass you created in this task by updating the
*change_me* field in [exercise3Task8-2.yaml](./exercise3Task8-2.yaml).

Save the file as **exercise3Task8-2mod.yaml** and then execute the file:


<details> <summary>Solution </summary> 

[exercise3Task8-2.yaml](./Solutions/exercise3Task8-2.yaml)

 ```yaml

In [ ]:
kubectl create -f exercise3Task8-2mod.yaml


---

---

Review the PVC and PV created by exercise3Task8-2.yaml.


---


Within ONTAP System Manager, locate the name of the volume created by exercise3Task8-2.yaml. 

Notice the naming structure of the volume and the qtrees.


In [ ]:
ssh admin@cluster1 volume show -vserver svm0|grep eco

---

Create another persistent volume claim in [exercise3Task8-3.yaml](./exercise3Task8-3.yaml):

<details> <summary>Solution </summary> 

[exercise3Task8-3.yaml](./Solutions/exercise3Task8-3.yaml)

 ```yaml


In [ ]:
kubectl create -f exercise3Task8-3mod.yaml


---

Review the resulting qtree in ONTAP System Manager.


In [ ]:
ssh admin@cluster1 qtree show -vserver svm0


#### Task 9: Provision NVMe namespaces using Trident

In this task, you create an NVMe back end and a storage class, and you use that storage class to
create a persistent volume for a pod. 

Previously, in Task 6, you configured svm1 to serve the NVMe/TCP protocol.


---

Update the [exercise3Task9-1.yaml](./exercise3Task9-1.yaml) file with the details of the NVMe functionality for svm1 by using the **ontap-san** storage driver:

Save the file as **exercise3Task9-1mod.yaml**

  - Management LIF: 192.168.0.40
  
  - SVM: svm1
  
  - sanType: nvme
  
  - useREST: true

NOTE:

You created the credentials secret, c1-svm1-backend-secret, previously.

<details> <summary>Solution </summary>

[exercise3Task9-1.yaml](./Solutions/exercise3Task9-1.yaml)  

```yaml

---


---

Create the secret and the back end by using the kubectl tool:


In [ ]:
kubectl create -f exercise3Task9-1mod.yaml


---

In the Kubernetes IDE extension, ensure you are in the trident namespace. 

Navigate to 

**Clusters** > 

  **source-admin@source** > 

**Custom Resources** >

**tridentbackendconfigs** > 

**c1-svm1-nvme-tbc**.

This back end is the one that you created. 

The status should show the last operation status as success and the phase as bound.


---

Verify that you created the back-end configuration:



In [ ]:
kubectl -n trident get tbc -o wide


---

Get details on the back-end configuration that you created:


In [ ]:
kubectl -n trident describe tbc c1-svm1-nvme-tbc


---

Review and update the name of the back end in the YAML, and then create the storage class in the exercise3Task9-2.yaml file:

Save the file as **exercise3Task9-2mod.yaml**

<details> <summary>Solution </summary> 

[exercise3Task9-2.yaml](./Solutions/exercise3Task9-2.yaml) 

```yaml

In [ ]:

kubectl create -f exercise3Task9-2mod.yaml


---

Review and update the storage class name in the YAML, and then create the PVC in the
[exercise3Task9-3.yaml](./exercise3Task9-3.yaml) file:

Save the file as exercise3Task9-3mod.yaml

<details> <summary>Solution </summary>

[exercise3Task9-3.yaml](./Solutions/exercise3Task9-3.yaml)  

```yaml

In [ ]:
kubectl create -f exercise3Task9-3mod.yaml


---

Navigate to ONTAP System Manager and see the new volume that Trident created:

https://192.168.0.101/sysmgr/v4/storage/volumes.


In [ ]:
ssh admin@cluster1 volume show -vserver svm1

---

Navigate to NVMe Namespaces in ONTAP System Manager and see the new namespace that
Trident created:

https://192.168.0.101/sysmgr/v4/storage/nvmeNamespaces.


In [ ]:
ssh admin@cluster1 nvme namespace show -vserver svm1 

---

Review and update the claim name in the YAML, and then create the pod in the
[exercise3Task9-4.yaml](./exercise3Task9-4.yaml) file:

Save the file as exercise3Task9-4mod.yaml

<details> <summary>Solution </summary>  

[exercise3Task9-4.yaml](./Solutions/exercise3Task9-4.yaml)

```yaml

In [ ]:

kubectl create -f exercise3Task9-4mod.yaml


---

Verify that the pod was created successfully:


In [ ]:

kubectl -n default get pod nvme-pod


---

CHALLENGE STEP: 

Expose the pod using a service and try to access it.



In [ ]:
kubectl -n default expose pod nvme-pod --type=NodePort --name=nvme-service


In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "echo '<html><body>Hello nfs-pod using NFS</body></html>' >  /usr/share/nginx/html/index.html;"


In [ ]:
kubectl -n default exec  -it nvme-pod -- /bin/sh -c "echo '<html><body>hello from nvme-pod </body></html>' > /usr/share/nginx/html/index.html"

In [ ]:
nodeport=$(kubectl -n default describe service nvme-service|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

---
---

End of exercise